### User의 history에 맞게 영화를 추천해 주기.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

In [4]:
movie_path = "movielens/movies.csv"
rate_path = "movielens/ratings.csv"

In [5]:
mv_data = pd.read_csv(movie_path)
rate_data = pd.read_csv(rate_path)

### Matrix (NxM) : N명의 사용자가 M개의 영화의 평점에 대한 행렬 

In [6]:
data = rate_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

In [7]:
data.head(5)

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
userId,,,,,,,,,,,,,,,,,,,,,
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [8]:
data[1]

userId
1      0.0
2      0.0
3      0.0
4      0.0
5      0.0
      ... 
667    0.0
668    0.0
669    0.0
670    4.0
671    5.0
Name: 1, Length: 671, dtype: float64

In [9]:
data.shape
## 671명의 사용자의 9066개 영화에 대한 평점 행렬.

(671, 9066)

In [10]:
data_mat = data.as_matrix()
data_mean = np.mean(data_mat, axis=1)

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.


In [11]:
data_mean.shape

(671,)

In [12]:
data_user_mean = data_mat-data_mean.reshape(-1,1)

In [13]:
data_user_mean.shape

(671, 9066)

## SVD

In [14]:
from scipy.sparse.linalg import svds
U, Sigma, Vtrans = svds(data_user_mean, k=12)

In [17]:
print("U shape = ", U.shape)
print("Sigma shape = ", Sigma.shape)
print("V transpose shape = ", Vtrans.shape)

U shape =  (671, 12)
Sigma shape =  (12,)
V transpose shape =  (12, 9066)


In [18]:
Sigma = np.diag(Sigma)
Sigma

array([[105.72437051,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        , 110.15522471,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        , 116.29714259,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        , 118.66831462,
          0.        ,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,   0.        ,   0.        ,
        122.82232277,   0.        ,   0.        ,   0.        ,
          0.        ,   0.        ,   0.        ,   0.        ],
       [  0.        ,   0.        ,

In [19]:
original = np.dot(np.dot(U,Sigma),Vtrans)
original

array([[-8.53118777e-02,  1.61536749e-02, -1.94620564e-02, ...,
        -1.12007085e-02, -1.09228322e-02, -9.39184267e-03],
       [ 1.39922174e+00,  1.57961114e+00,  5.00245651e-01, ...,
        -3.01873474e-02, -2.99832308e-02, -2.32941396e-03],
       [ 9.57171301e-01,  3.76896368e-01, -1.97763245e-02, ...,
        -5.43804303e-03, -6.78783733e-03, -2.58158816e-02],
       ...,
       [ 3.47026262e-01, -1.36938465e-02,  7.25834122e-02, ...,
        -1.50174709e-02, -1.48668301e-02, -1.86230070e-02],
       [ 1.04150059e+00,  2.52063513e-01,  2.10766547e-01, ...,
        -7.67203008e-03, -8.08771808e-03, -2.14991642e-02],
       [ 2.69928616e+00,  1.39848659e-01,  1.36485141e-01, ...,
        -2.92447044e-02, -3.10765518e-02, -6.91201222e-02]])

In [20]:
# 평균 값을 빼고 연산했으니 이제는 더해준다.
svd_mat = original+data_mean.reshape(-1,1)

In [21]:
svd_mat, svd_mat.shape

(array([[-7.96864641e-02,  2.17790885e-02, -1.38366428e-02, ...,
         -5.57529490e-03, -5.29741854e-03, -3.76642904e-03],
        [ 1.42845183e+00,  1.60884123e+00,  5.29475741e-01, ...,
         -9.57256957e-04, -7.53140349e-04,  2.69006765e-02],
        [ 9.77246307e-01,  3.96971374e-01,  2.98681042e-04, ...,
          1.46369625e-02,  1.32871682e-02, -5.74087610e-03],
        ...,
        [ 3.60703739e-01, -1.63702536e-05,  8.62608885e-02, ...,
         -1.33999466e-03, -1.18935380e-03, -4.94553070e-03],
        [ 1.05451625e+00,  2.65079176e-01,  2.23782210e-01, ...,
          5.34363284e-03,  4.92794484e-03, -8.48350126e-03],
        [ 2.74897731e+00,  1.89539813e-01,  1.86176295e-01, ...,
          2.04464494e-02,  1.86146020e-02, -1.94289684e-02]]), (671, 9066))

In [22]:
df_svd_mat = pd.DataFrame(svd_mat, columns=data.columns)

In [23]:
df_svd_mat.head()

movieId,1,2,3,4,5,6,7,8,9,10,...,161084,161155,161594,161830,161918,161944,162376,162542,162672,163949
0,-0.079686,0.021779,-0.013837,-0.005870,-0.028877,0.032371,0.000715,-0.004428,-0.005219,0.038195,...,-0.004324,-0.004352,0.010478,-0.004256,-0.003944,-0.005674,0.018157,-0.005575,-0.005297,-0.003766
1,1.428452,1.608841,0.529476,0.168278,0.520809,1.107473,0.529719,0.089376,0.296270,1.970031,...,0.013227,-0.002275,0.020680,-0.005245,-0.007644,-0.021019,0.031243,-0.000957,-0.000753,0.026901
2,0.977246,0.396971,0.000299,0.027444,0.021287,0.141458,-0.057134,0.031633,-0.012538,0.383576,...,0.002761,0.004907,-0.014190,-0.000251,-0.006007,-0.003189,-0.026916,0.014637,0.013287,-0.005741
3,1.870844,1.169993,0.252202,0.094831,-0.181713,-0.511953,-0.027820,-0.143080,0.013247,1.461694,...,0.026412,-0.027245,0.054681,0.018450,0.034544,-0.035740,0.088889,-0.019365,-0.017113,0.066559
4,1.182777,0.924903,0.075998,0.061505,0.602680,-0.159825,0.339925,0.081534,-0.079666,0.535018,...,-0.029124,-0.029357,0.009064,-0.029092,-0.030890,-0.057453,0.026344,-0.024027,-0.024614,-0.032752


In [30]:
def recommand_Search(userID):
    userID_idx=userID-1
    user_mat = df_svd_mat.iloc[userID_idx].sort_values(ascending=False)
    user_data = rate_data[rate_data.userId==userID]
    user_history = user_data.merge(mv_data, on="movieId").sort_values(["rating"], ascending=False)
    recomm = mv_data[~mv_data["movieId"].isin(user_history["userId"])]
    recomm = recomm.merge(pd.DataFrame(user_mat).reset_index(), on="movieId")
    recomm = recomm.rename(columns={userID_idx: "Predictions"})
    recomm = recomm.sort_values(["Predictions"], ascending=False)
    return user_history, recomm

In [38]:
his, recommand = recommand_Search(250)

In [39]:
his.head(5)

,userId,movieId,rating,timestamp,title,genres
154,250,152081,5.0,1469700464,Zootopia (2016),Action|Adventure|Animation|Children|Comedy
38,250,4973,5.0,1469700716,"Amelie (Fabuleux destin d'Amélie Poulain, Le) ...",Comedy|Romance
102,250,60069,5.0,1469807452,WALL·E (2008),Adventure|Animation|Children|Romance|Sci-Fi
55,250,33615,5.0,1469807586,Madagascar (2005),Adventure|Animation|Children|Comedy
99,250,59784,5.0,1469807361,Kung Fu Panda (2008),Action|Animation|Children|Comedy|IMAX


In [40]:
recommand.head(5)

,movieId,title,genres,Predictions
6891,58559,"Dark Knight, The (2008)",Action|Crime|Drama|IMAX,4.492494
7550,79132,Inception (2010),Action|Crime|Drama|Mystery|Sci-Fi|Thriller|IMAX,4.114310
2373,2959,Fight Club (1999),Action|Crime|Drama|Thriller,3.846712
283,318,"Shawshank Redemption, The (1994)",Crime|Drama,3.770221
320,356,Forrest Gump (1994),Comedy|Drama|Romance|War,3.744284
